#Step 1: LLM from Hugging Face

Firstly, I am going to import all the libraries required for our project. In parallel, I will explain their uses.

In [1]:
! pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [2]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00


In [10]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

I am going to use AutoTokenizer for tokeninzing and AutoModelForCausalLM use for load pretrained model from HuggingFace and it will also help to predict next word

the Salesforce/codegen-350M-mono is used for genrate the code, fix the error, learning documentation and code translation

torch.device("cuda" if torch.cuda.is_available() else "cpu") in this line of i am checking for GPU. if GPU available then use it other wise use CPU.

In [4]:
model_name = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

In this line of code we enter the input as text then convert it as token. return_tensors = "pt" give the output as tensor.

outputs = model.generate(**inputs, max_length=100)

it genrate the code and as we can see limited the token size 100. So it will give responce under the limit.

return tokenizer.decode(outputs[0], skip_special_tokens=True)

this will return the the string in human readable format and skip_special_tokens=True will remove padding and all that stuff.

In [5]:
def generate_code(prompt: str, model, tokenizer, device):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

#Step 2: Assistant Agents

Here i am defining a mothod then generate the responce based on prompt. Using generate_code to generateb the responce though passing the prompt, model, tokeniztion.

Now Instantiates two agents with the names User and Assistant.

In [6]:
class Agent:
    def __init__(self, name):
        self.name = name

    def respond(self, prompt, model, tokenizer, device):
        return generate_code(prompt, model, tokenizer, device)

user_agent = Agent("User")
assistant_agent = Agent("Assistant")

#Step 3:  Function Calls to Agent Chat

Here function define shows interaction between user and Assistance.

In [7]:
def chat(user_input, user_agent, assistant_agent, model, tokenizer, device):
    print(f"{user_agent.name}: {user_input}")
    response = assistant_agent.respond(user_input, model, tokenizer, device)
    print(f"{assistant_agent.name}: {response}")
    return response

#Step 4: Implementing RetrieveAgents with RAG for QA and Code

embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

Here pretrained model is use for generate document embedding. sentence-transformers/all-MiniLM-L6-v2 this model trained for Document embedding.

documents = ["Document 1 content about Canoo.", "Document 2 content about another topic."]

this line of code is use for document embedding.Assistant can provide better responses by understanding the context in which the code should be generated. if a user asks for code to "add two numbers," the system might retrieve a relevant document discussing basic arithmetic operations in Python and then generate the code

vector_store = FAISS.from_texts(texts=documents, embedding=hf_embeddings)

vector database is used for index the embedding. it is helpful for similary search and easy to find relavant info.

In [11]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
documents = ["Python Operations.", "Python advanced technique."]
vector_store = FAISS.from_texts(texts=documents, embedding=hf_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

results = vector_store.similarity_search(query, k=1)

As i told here we i used similarity search for most relavant doc. K=1 indicate to fetch top 1 doc.

In [12]:
def retrieve_context(query, vector_store):
    results = vector_store.similarity_search(query, k=1)
    return results[0] if results else "No relevant document found."

This function increase the chat experience by  conetextual info to generate more relevant and helpful responses from the assistant agent.

In [13]:
def chat_with_context(user_input, user_agent, assistant_agent, model, tokenizer, device, vector_store):
    print(f"{user_agent.name}: {user_input}")
    context = retrieve_context(user_input, vector_store)
    augmented_prompt = f"{context}\n\n{user_input}"
    response = assistant_agent.respond(augmented_prompt, model, tokenizer, device)
    print(f"{assistant_agent.name}: {response}")
    return response

In [15]:
user_input = "Add two number in python"
chat_with_context(user_input, user_agent, assistant_agent, model, tokenizer, device, vector_store)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: Add two number in python
Assistant: page_content='Python Operations.'

Add two number in python

def add(x,y):
    return x+y

def sub(x,y):
    return x-y

def mul(x,y):
    return x*y

def div(x,y):
    return x/y

print(add(10,20))
print(sub(10,20))
print(mul(10


"page_content='Python Operations.'\n\nAdd two number in python\n\ndef add(x,y):\n    return x+y\n\ndef sub(x,y):\n    return x-y\n\ndef mul(x,y):\n    return x*y\n\ndef div(x,y):\n    return x/y\n\nprint(add(10,20))\nprint(sub(10,20))\nprint(mul(10"